In [1]:
import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import collections
from collections import OrderedDict
%matplotlib inline
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
####### Macro Cases ######
dfn8_macro_fused_probempi = pd.read_csv("n8_fused_macro_probmpi.csv")
dfn8_macro_fused_nonprobempi = pd.read_csv("n8_fused_macro_nonprobmpi.csv")
dfn8_macro_newgeo_probempi = pd.read_csv("n8_newgeo_macro_probmpi.csv")
dfn8_macro_newgeo_nonprobempi = pd.read_csv("n8_newgeo_macro_nonprobmpi.csv")
dfn8_macro_nonfused_probempi = pd.read_csv("n8_nonfused_macro_probmpi.csv")
dfn8_macro_nonfused_nonprobempi = pd.read_csv("n8_nonfused_macro_nonprobmpi.csv")

####### AVX Cases ######
dfn8_avx_fused_probmpi = pd.read_csv("n8_fused_avx_probmpi.csv")
# dfn8_avx_fused_probmpi = pd.read_csv("n8_fused_avx_probmpi_updated.csv")
dfn8_avx_fused_nonprobmpi = pd.read_csv("n8_fused_avx_nonprobmpi.csv")
# dfn8_avx_newgeo_probempi = pd.read_csv("n4_newgeo_avx_probmpi.csv")
# dfn8_avx_newgeo_nonprobempi = pd.read_csv("n4_newgeo_avx_nonprobmpi.csv")
dfn8_avx_nonfused_probempi = pd.read_csv("n8_nonfused_avx_probmpi.csv")
dfn8_avx_nonfused_nonprobempi = pd.read_csv("n8_nonfused_avx_nonprobmpi.csv")

####### VECT Cases ######




# MACRO CASES

## 1) Fused_Macro_Probempi

### Adding Columns

In [3]:
dfn8_macro_fused_probempi["N_Nodes"] = 8
dfn8_macro_fused_probempi['NPX'] = (dfn8_macro_fused_probempi.GLB_X//dfn8_macro_fused_probempi.LOC_X)
dfn8_macro_fused_probempi['NPY'] = (dfn8_macro_fused_probempi.GLB_Y//dfn8_macro_fused_probempi.LOC_Y)
dfn8_macro_fused_probempi['NPZ'] = (dfn8_macro_fused_probempi.GLB_Z//dfn8_macro_fused_probempi.LOC_Z)
dfn8_macro_fused_probempi['NPT'] = (dfn8_macro_fused_probempi.GLB_T//dfn8_macro_fused_probempi.LOC_T)
dfn8_macro_fused_probempi["Total_proc"] = dfn8_macro_fused_probempi["omp_thrd"]* dfn8_macro_fused_probempi.MPI_size
# dfn8_macro_fused_probempi.columns

### Data Summary

In [4]:
dfn8_macro_fused_probempi_summary = dfn8_macro_fused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_macro_fused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,137
1,72,14,9,2,2,2,1008,62
2,112,9,14,2,2,2,1008,77
3,128,8,16,2,2,2,1024,70
4,144,7,18,2,2,2,1008,60
5,168,6,21,2,2,2,1008,25
6,200,5,25,2,2,2,1000,21
7,248,4,31,2,2,2,992,18
8,256,4,32,2,2,2,1024,35
9,328,3,41,2,2,2,984,13


### Cases Setup

In [5]:
dfn8MacFusProb_64mpi_16omp = dfn8_macro_fused_probempi[(dfn8_macro_fused_probempi.omp_thrd == 16) & (dfn8_macro_fused_probempi.GLB_T/dfn8_macro_fused_probempi.LOC_T == 8) & (dfn8_macro_fused_probempi.GLB_X/dfn8_macro_fused_probempi.LOC_X == 2) & (dfn8_macro_fused_probempi.GLB_Y/dfn8_macro_fused_probempi.LOC_Y == 2)& (dfn8_macro_fused_probempi.GLB_Z/dfn8_macro_fused_probempi.LOC_Z == 2)]
dfn8MacFusProb_128mpi_8omp = dfn8_macro_fused_probempi[(dfn8_macro_fused_probempi.omp_thrd == 8) & (dfn8_macro_fused_probempi.GLB_T/dfn8_macro_fused_probempi.LOC_T == 16) & (dfn8_macro_fused_probempi.GLB_X/dfn8_macro_fused_probempi.LOC_X == 2) & (dfn8_macro_fused_probempi.GLB_Y/dfn8_macro_fused_probempi.LOC_Y == 2)& (dfn8_macro_fused_probempi.GLB_Z/dfn8_macro_fused_probempi.LOC_Z == 2)]
dfn8MacFusProb_256mpi_4omp = dfn8_macro_fused_probempi[(dfn8_macro_fused_probempi.omp_thrd == 4) & (dfn8_macro_fused_probempi.GLB_T/dfn8_macro_fused_probempi.LOC_T == 32) & (dfn8_macro_fused_probempi.GLB_X/dfn8_macro_fused_probempi.LOC_X == 2) & (dfn8_macro_fused_probempi.GLB_Y/dfn8_macro_fused_probempi.LOC_Y == 2)& (dfn8_macro_fused_probempi.GLB_Z/dfn8_macro_fused_probempi.LOC_Z == 2)]
dfn8MacFusProb_512mpi_2omp = dfn8_macro_fused_probempi[(dfn8_macro_fused_probempi.omp_thrd == 2) & (dfn8_macro_fused_probempi.GLB_T/dfn8_macro_fused_probempi.LOC_T == 64) & (dfn8_macro_fused_probempi.GLB_X/dfn8_macro_fused_probempi.LOC_X == 2) & (dfn8_macro_fused_probempi.GLB_Y/dfn8_macro_fused_probempi.LOC_Y == 2)& (dfn8_macro_fused_probempi.GLB_Z/dfn8_macro_fused_probempi.LOC_Z == 2)]
dfn8MacFusProb_1024mpi_1omp = dfn8_macro_fused_probempi[(dfn8_macro_fused_probempi.omp_thrd == 1) & (dfn8_macro_fused_probempi.GLB_T/dfn8_macro_fused_probempi.LOC_T == 128) & (dfn8_macro_fused_probempi.GLB_X/dfn8_macro_fused_probempi.LOC_X == 2) & (dfn8_macro_fused_probempi.GLB_Y/dfn8_macro_fused_probempi.LOC_Y == 2)& (dfn8_macro_fused_probempi.GLB_Z/dfn8_macro_fused_probempi.LOC_Z == 2)]
dfn8MacFusProb_168mpi_6omp = dfn8_macro_fused_probempi[(dfn8_macro_fused_probempi.omp_thrd == 6) & (dfn8_macro_fused_probempi.GLB_T/dfn8_macro_fused_probempi.LOC_T == 21) & (dfn8_macro_fused_probempi.GLB_X/dfn8_macro_fused_probempi.LOC_X == 2) & (dfn8_macro_fused_probempi.GLB_Y/dfn8_macro_fused_probempi.LOC_Y == 2)& (dfn8_macro_fused_probempi.GLB_Z/dfn8_macro_fused_probempi.LOC_Z == 2)]
dfn8MacFusProb_504mpi_2omp = dfn8_macro_fused_probempi[(dfn8_macro_fused_probempi.omp_thrd == 2) & (dfn8_macro_fused_probempi.GLB_T/dfn8_macro_fused_probempi.LOC_T == 63) & (dfn8_macro_fused_probempi.GLB_X/dfn8_macro_fused_probempi.LOC_X == 2) & (dfn8_macro_fused_probempi.GLB_Y/dfn8_macro_fused_probempi.LOC_Y == 2)& (dfn8_macro_fused_probempi.GLB_Z/dfn8_macro_fused_probempi.LOC_Z == 2)]


## 2) Fused_Macro_nonprobempi

### Adding Columns

In [6]:
dfn8_macro_fused_nonprobempi["N_Nodes"] = 8
dfn8_macro_fused_nonprobempi['NPX'] = (dfn8_macro_fused_nonprobempi.GLB_X//dfn8_macro_fused_nonprobempi.LOC_X)
dfn8_macro_fused_nonprobempi['NPY'] = (dfn8_macro_fused_nonprobempi.GLB_Y//dfn8_macro_fused_nonprobempi.LOC_Y)
dfn8_macro_fused_nonprobempi['NPZ'] = (dfn8_macro_fused_nonprobempi.GLB_Z//dfn8_macro_fused_nonprobempi.LOC_Z)
dfn8_macro_fused_nonprobempi['NPT'] = (dfn8_macro_fused_nonprobempi.GLB_T//dfn8_macro_fused_nonprobempi.LOC_T)
dfn8_macro_fused_nonprobempi["Total_proc"] = dfn8_macro_fused_nonprobempi["omp_thrd"]* dfn8_macro_fused_nonprobempi.MPI_size
# dfn8_macro_fused_nonprobempi.columns

### Data Summary

In [7]:
dfn8_macro_fused_nonprobempi_summary = dfn8_macro_fused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_macro_fused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,140
1,72,14,9,2,2,2,1008,60
2,112,9,14,2,2,2,1008,79
3,128,8,16,2,2,2,1024,68
4,144,7,18,2,2,2,1008,61
5,168,6,21,2,2,2,1008,26
6,200,5,25,2,2,2,1000,21
7,248,4,31,2,2,2,992,18
8,256,4,32,2,2,2,1024,35
9,328,3,41,2,2,2,984,12


### Cases Setup

In [8]:
dfn8MacFusNonProb_64mpi_16omp = dfn8_macro_fused_nonprobempi[(dfn8_macro_fused_nonprobempi.omp_thrd == 16) & (dfn8_macro_fused_nonprobempi.GLB_T/dfn8_macro_fused_nonprobempi.LOC_T == 8) & (dfn8_macro_fused_nonprobempi.GLB_X/dfn8_macro_fused_nonprobempi.LOC_X == 2) & (dfn8_macro_fused_nonprobempi.GLB_Y/dfn8_macro_fused_nonprobempi.LOC_Y == 2)& (dfn8_macro_fused_nonprobempi.GLB_Z/dfn8_macro_fused_nonprobempi.LOC_Z == 2)]
dfn8MacFusNonProb_128mpi_8omp = dfn8_macro_fused_nonprobempi[(dfn8_macro_fused_nonprobempi.omp_thrd == 8) & (dfn8_macro_fused_nonprobempi.GLB_T/dfn8_macro_fused_nonprobempi.LOC_T == 16) & (dfn8_macro_fused_nonprobempi.GLB_X/dfn8_macro_fused_nonprobempi.LOC_X == 2) & (dfn8_macro_fused_nonprobempi.GLB_Y/dfn8_macro_fused_nonprobempi.LOC_Y == 2)& (dfn8_macro_fused_nonprobempi.GLB_Z/dfn8_macro_fused_nonprobempi.LOC_Z == 2)]
dfn8MacFusNonProb_256mpi_4omp = dfn8_macro_fused_nonprobempi[(dfn8_macro_fused_nonprobempi.omp_thrd == 4) & (dfn8_macro_fused_nonprobempi.GLB_T/dfn8_macro_fused_nonprobempi.LOC_T == 32) & (dfn8_macro_fused_nonprobempi.GLB_X/dfn8_macro_fused_nonprobempi.LOC_X == 2) & (dfn8_macro_fused_nonprobempi.GLB_Y/dfn8_macro_fused_nonprobempi.LOC_Y == 2)& (dfn8_macro_fused_nonprobempi.GLB_Z/dfn8_macro_fused_nonprobempi.LOC_Z == 2)]
dfn8MacFusNonProb_512mpi_2omp = dfn8_macro_fused_nonprobempi[(dfn8_macro_fused_nonprobempi.omp_thrd == 2) & (dfn8_macro_fused_nonprobempi.GLB_T/dfn8_macro_fused_nonprobempi.LOC_T == 64) & (dfn8_macro_fused_nonprobempi.GLB_X/dfn8_macro_fused_nonprobempi.LOC_X == 2) & (dfn8_macro_fused_nonprobempi.GLB_Y/dfn8_macro_fused_nonprobempi.LOC_Y == 2)& (dfn8_macro_fused_nonprobempi.GLB_Z/dfn8_macro_fused_nonprobempi.LOC_Z == 2)]
dfn8MacFusNonProb_1024mpi_1omp = dfn8_macro_fused_nonprobempi[(dfn8_macro_fused_nonprobempi.omp_thrd == 1) & (dfn8_macro_fused_nonprobempi.GLB_T/dfn8_macro_fused_nonprobempi.LOC_T == 128) & (dfn8_macro_fused_nonprobempi.GLB_X/dfn8_macro_fused_nonprobempi.LOC_X == 2) & (dfn8_macro_fused_nonprobempi.GLB_Y/dfn8_macro_fused_nonprobempi.LOC_Y == 2)& (dfn8_macro_fused_nonprobempi.GLB_Z/dfn8_macro_fused_nonprobempi.LOC_Z == 2)]
dfn8MacFusNonProb_168mpi_6omp = dfn8_macro_fused_nonprobempi[(dfn8_macro_fused_nonprobempi.omp_thrd == 6) & (dfn8_macro_fused_nonprobempi.GLB_T/dfn8_macro_fused_nonprobempi.LOC_T == 21) & (dfn8_macro_fused_nonprobempi.GLB_X/dfn8_macro_fused_nonprobempi.LOC_X == 2) & (dfn8_macro_fused_nonprobempi.GLB_Y/dfn8_macro_fused_nonprobempi.LOC_Y == 2)& (dfn8_macro_fused_nonprobempi.GLB_Z/dfn8_macro_fused_nonprobempi.LOC_Z == 2)]
dfn8MacFusNonProb_504mpi_2omp = dfn8_macro_fused_nonprobempi[(dfn8_macro_fused_nonprobempi.omp_thrd == 2) & (dfn8_macro_fused_nonprobempi.GLB_T/dfn8_macro_fused_nonprobempi.LOC_T == 63) & (dfn8_macro_fused_nonprobempi.GLB_X/dfn8_macro_fused_nonprobempi.LOC_X == 2) & (dfn8_macro_fused_nonprobempi.GLB_Y/dfn8_macro_fused_nonprobempi.LOC_Y == 2)& (dfn8_macro_fused_nonprobempi.GLB_Z/dfn8_macro_fused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Fused Macro: Probempi vs NonProbempi

In [9]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacFusProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_1024mpi_1omp.GFLOPS*(dfn8MacFusProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_1024mpi_1omp.GFLOPS*(dfn8MacFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacFusProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_512mpi_2omp.GFLOPS*(dfn8MacFusProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacFusNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_512mpi_2omp.GFLOPS*(dfn8MacFusNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacFusProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_256mpi_4omp.GFLOPS*(dfn8MacFusProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacFusNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_256mpi_4omp.GFLOPS*(dfn8MacFusNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacFusProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_128mpi_8omp.GFLOPS*(dfn8MacFusProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacFusNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_128mpi_8omp.GFLOPS*(dfn8MacFusNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacFusProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_64mpi_16omp.GFLOPS*(dfn8MacFusProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacFusNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_64mpi_16omp.GFLOPS*(dfn8MacFusNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacFusProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_168mpi_6omp.GFLOPS*(dfn8MacFusProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacFusNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_168mpi_6omp.GFLOPS*(dfn8MacFusNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacFusProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_504mpi_2omp.GFLOPS*(dfn8MacFusProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacFusNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_504mpi_2omp.GFLOPS*(dfn8MacFusNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Fused-Macro (8-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Fused Macro: Probempi vs NonProbempi

In [10]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacFusProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_1024mpi_1omp.BAND*(dfn8MacFusProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_1024mpi_1omp.BAND*(dfn8MacFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacFusProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_512mpi_2omp.BAND*(dfn8MacFusProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacFusNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_512mpi_2omp.BAND*(dfn8MacFusNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacFusProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_256mpi_4omp.BAND*(dfn8MacFusProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacFusNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_256mpi_4omp.BAND*(dfn8MacFusNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacFusProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_128mpi_8omp.BAND*(dfn8MacFusProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacFusNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_128mpi_8omp.BAND*(dfn8MacFusNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacFusProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_64mpi_16omp.BAND*(dfn8MacFusProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacFusNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_64mpi_16omp.BAND*(dfn8MacFusNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacFusProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_168mpi_6omp.BAND*(dfn8MacFusProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacFusNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_168mpi_6omp.BAND*(dfn8MacFusNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8MacFusProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacFusProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusProb_504mpi_2omp.BAND*(dfn8MacFusProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacFusNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacFusNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacFusNonProb_504mpi_2omp.BAND*(dfn8MacFusNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacFusNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))

# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Fused Macro: Probempi vs NonProbempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 






## 3) Nonfused_Macro_Probempi

### Adding Columns

In [11]:
dfn8_macro_nonfused_probempi["N_Nodes"] = 8
dfn8_macro_nonfused_probempi['NPX'] = (dfn8_macro_nonfused_probempi.GLB_X//dfn8_macro_nonfused_probempi.LOC_X)
dfn8_macro_nonfused_probempi['NPY'] = (dfn8_macro_nonfused_probempi.GLB_Y//dfn8_macro_nonfused_probempi.LOC_Y)
dfn8_macro_nonfused_probempi['NPZ'] = (dfn8_macro_nonfused_probempi.GLB_Z//dfn8_macro_nonfused_probempi.LOC_Z)
dfn8_macro_nonfused_probempi['NPT'] = (dfn8_macro_nonfused_probempi.GLB_T//dfn8_macro_nonfused_probempi.LOC_T)
dfn8_macro_nonfused_probempi["Total_proc"] = dfn8_macro_nonfused_probempi["omp_thrd"]* dfn8_macro_nonfused_probempi.MPI_size
# dfn8_macro_nonfused_probempi.columns

### Data Summary

In [12]:
dfn8_macro_nonfused_probempi_summary = dfn8_macro_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_macro_nonfused_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,97
1,72,14,9,2,2,2,1008,39
2,112,9,14,2,2,2,1008,77
3,128,8,16,2,2,2,1024,68
4,144,7,18,2,2,2,1008,58
5,168,6,21,2,2,2,1008,24
6,200,5,25,2,2,2,1000,22
7,248,4,31,2,2,2,992,18
8,256,4,32,2,2,2,1024,34
9,328,3,41,2,2,2,984,12


### Cases Setup

In [13]:
dfn8MacNonFusProb_64mpi_16omp = dfn8_macro_nonfused_probempi[(dfn8_macro_nonfused_probempi.omp_thrd == 16) & (dfn8_macro_nonfused_probempi.GLB_T/dfn8_macro_nonfused_probempi.LOC_T == 8) & (dfn8_macro_nonfused_probempi.GLB_X/dfn8_macro_nonfused_probempi.LOC_X == 2) & (dfn8_macro_nonfused_probempi.GLB_Y/dfn8_macro_nonfused_probempi.LOC_Y == 2)& (dfn8_macro_nonfused_probempi.GLB_Z/dfn8_macro_nonfused_probempi.LOC_Z == 2)]
dfn8MacNonFusProb_128mpi_8omp = dfn8_macro_nonfused_probempi[(dfn8_macro_nonfused_probempi.omp_thrd == 8) & (dfn8_macro_nonfused_probempi.GLB_T/dfn8_macro_nonfused_probempi.LOC_T == 16) & (dfn8_macro_nonfused_probempi.GLB_X/dfn8_macro_nonfused_probempi.LOC_X == 2) & (dfn8_macro_nonfused_probempi.GLB_Y/dfn8_macro_nonfused_probempi.LOC_Y == 2)& (dfn8_macro_nonfused_probempi.GLB_Z/dfn8_macro_nonfused_probempi.LOC_Z == 2)]
dfn8MacNonFusProb_256mpi_4omp = dfn8_macro_nonfused_probempi[(dfn8_macro_nonfused_probempi.omp_thrd == 4) & (dfn8_macro_nonfused_probempi.GLB_T/dfn8_macro_nonfused_probempi.LOC_T == 32) & (dfn8_macro_nonfused_probempi.GLB_X/dfn8_macro_nonfused_probempi.LOC_X == 2) & (dfn8_macro_nonfused_probempi.GLB_Y/dfn8_macro_nonfused_probempi.LOC_Y == 2)& (dfn8_macro_nonfused_probempi.GLB_Z/dfn8_macro_nonfused_probempi.LOC_Z == 2)]
dfn8MacNonFusProb_512mpi_2omp = dfn8_macro_nonfused_probempi[(dfn8_macro_nonfused_probempi.omp_thrd == 2) & (dfn8_macro_nonfused_probempi.GLB_T/dfn8_macro_nonfused_probempi.LOC_T == 64) & (dfn8_macro_nonfused_probempi.GLB_X/dfn8_macro_nonfused_probempi.LOC_X == 2) & (dfn8_macro_nonfused_probempi.GLB_Y/dfn8_macro_nonfused_probempi.LOC_Y == 2)& (dfn8_macro_nonfused_probempi.GLB_Z/dfn8_macro_nonfused_probempi.LOC_Z == 2)]
dfn8MacNonFusProb_1024mpi_1omp = dfn8_macro_nonfused_probempi[(dfn8_macro_nonfused_probempi.omp_thrd == 1) & (dfn8_macro_nonfused_probempi.GLB_T/dfn8_macro_nonfused_probempi.LOC_T == 128) & (dfn8_macro_nonfused_probempi.GLB_X/dfn8_macro_nonfused_probempi.LOC_X == 2) & (dfn8_macro_nonfused_probempi.GLB_Y/dfn8_macro_nonfused_probempi.LOC_Y == 2)& (dfn8_macro_nonfused_probempi.GLB_Z/dfn8_macro_nonfused_probempi.LOC_Z == 2)]
dfn8MacNonFusProb_168mpi_6omp = dfn8_macro_nonfused_probempi[(dfn8_macro_nonfused_probempi.omp_thrd == 6) & (dfn8_macro_nonfused_probempi.GLB_T/dfn8_macro_nonfused_probempi.LOC_T == 21) & (dfn8_macro_nonfused_probempi.GLB_X/dfn8_macro_nonfused_probempi.LOC_X == 2) & (dfn8_macro_nonfused_probempi.GLB_Y/dfn8_macro_nonfused_probempi.LOC_Y == 2)& (dfn8_macro_nonfused_probempi.GLB_Z/dfn8_macro_nonfused_probempi.LOC_Z == 2)]
dfn8MacNonFusProb_504mpi_2omp = dfn8_macro_nonfused_probempi[(dfn8_macro_nonfused_probempi.omp_thrd == 2) & (dfn8_macro_nonfused_probempi.GLB_T/dfn8_macro_nonfused_probempi.LOC_T == 63) & (dfn8_macro_nonfused_probempi.GLB_X/dfn8_macro_nonfused_probempi.LOC_X == 2) & (dfn8_macro_nonfused_probempi.GLB_Y/dfn8_macro_nonfused_probempi.LOC_Y == 2)& (dfn8_macro_nonfused_probempi.GLB_Z/dfn8_macro_nonfused_probempi.LOC_Z == 2)]


## 4) Nonfused_Macro_NonProbempi

### Adding Columns

In [14]:
dfn8_macro_nonfused_nonprobempi["N_Nodes"] = 8
dfn8_macro_nonfused_nonprobempi['NPX'] = (dfn8_macro_nonfused_nonprobempi.GLB_X//dfn8_macro_nonfused_nonprobempi.LOC_X)
dfn8_macro_nonfused_nonprobempi['NPY'] = (dfn8_macro_nonfused_nonprobempi.GLB_Y//dfn8_macro_nonfused_nonprobempi.LOC_Y)
dfn8_macro_nonfused_nonprobempi['NPZ'] = (dfn8_macro_nonfused_nonprobempi.GLB_Z//dfn8_macro_nonfused_nonprobempi.LOC_Z)
dfn8_macro_nonfused_nonprobempi['NPT'] = (dfn8_macro_nonfused_nonprobempi.GLB_T//dfn8_macro_nonfused_nonprobempi.LOC_T)
dfn8_macro_nonfused_nonprobempi["Total_proc"] = dfn8_macro_nonfused_nonprobempi["omp_thrd"]* dfn8_macro_nonfused_nonprobempi.MPI_size
# dfn8_macro_nonfused_nonprobempi.columns

### Data Summary

In [15]:
dfn8_macro_nonfused_nonprobempi_summary = dfn8_macro_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_macro_nonfused_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,98
1,72,14,9,2,2,2,1008,38
2,112,9,14,2,2,2,1008,80
3,128,8,16,2,2,2,1024,68
4,144,7,18,2,2,2,1008,61
5,168,6,21,2,2,2,1008,24
6,200,5,25,2,2,2,1000,21
7,248,4,31,2,2,2,992,18
8,256,4,32,2,2,2,1024,35
9,328,3,41,2,2,2,984,13


### Cases Setup

In [16]:
dfn8MacNonFusNonProb_64mpi_16omp = dfn8_macro_nonfused_nonprobempi[(dfn8_macro_nonfused_nonprobempi.omp_thrd == 16) & (dfn8_macro_nonfused_nonprobempi.GLB_T/dfn8_macro_nonfused_nonprobempi.LOC_T == 8) & (dfn8_macro_nonfused_nonprobempi.GLB_X/dfn8_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_Y/dfn8_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_macro_nonfused_nonprobempi.GLB_Z/dfn8_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn8MacNonFusNonProb_128mpi_8omp = dfn8_macro_nonfused_nonprobempi[(dfn8_macro_nonfused_nonprobempi.omp_thrd == 8) & (dfn8_macro_nonfused_nonprobempi.GLB_T/dfn8_macro_nonfused_nonprobempi.LOC_T == 16) & (dfn8_macro_nonfused_nonprobempi.GLB_X/dfn8_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_Y/dfn8_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_macro_nonfused_nonprobempi.GLB_Z/dfn8_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn8MacNonFusNonProb_256mpi_4omp = dfn8_macro_nonfused_nonprobempi[(dfn8_macro_nonfused_nonprobempi.omp_thrd == 4) & (dfn8_macro_nonfused_nonprobempi.GLB_T/dfn8_macro_nonfused_nonprobempi.LOC_T == 32) & (dfn8_macro_nonfused_nonprobempi.GLB_X/dfn8_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_Y/dfn8_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_macro_nonfused_nonprobempi.GLB_Z/dfn8_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn8MacNonFusNonProb_512mpi_2omp = dfn8_macro_nonfused_nonprobempi[(dfn8_macro_nonfused_nonprobempi.omp_thrd == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_T/dfn8_macro_nonfused_nonprobempi.LOC_T == 64) & (dfn8_macro_nonfused_nonprobempi.GLB_X/dfn8_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_Y/dfn8_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_macro_nonfused_nonprobempi.GLB_Z/dfn8_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn8MacNonFusNonProb_1024mpi_1omp = dfn8_macro_nonfused_nonprobempi[(dfn8_macro_nonfused_nonprobempi.omp_thrd == 1) & (dfn8_macro_nonfused_nonprobempi.GLB_T/dfn8_macro_nonfused_nonprobempi.LOC_T == 128) & (dfn8_macro_nonfused_nonprobempi.GLB_X/dfn8_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_Y/dfn8_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_macro_nonfused_nonprobempi.GLB_Z/dfn8_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn8MacNonFusNonProb_168mpi_6omp = dfn8_macro_nonfused_nonprobempi[(dfn8_macro_nonfused_nonprobempi.omp_thrd == 6) & (dfn8_macro_nonfused_nonprobempi.GLB_T/dfn8_macro_nonfused_nonprobempi.LOC_T == 21) & (dfn8_macro_nonfused_nonprobempi.GLB_X/dfn8_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_Y/dfn8_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_macro_nonfused_nonprobempi.GLB_Z/dfn8_macro_nonfused_nonprobempi.LOC_Z == 2)]
dfn8MacNonFusNonProb_504mpi_2omp = dfn8_macro_nonfused_nonprobempi[(dfn8_macro_nonfused_nonprobempi.omp_thrd == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_T/dfn8_macro_nonfused_nonprobempi.LOC_T == 63) & (dfn8_macro_nonfused_nonprobempi.GLB_X/dfn8_macro_nonfused_nonprobempi.LOC_X == 2) & (dfn8_macro_nonfused_nonprobempi.GLB_Y/dfn8_macro_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_macro_nonfused_nonprobempi.GLB_Z/dfn8_macro_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - Nonfused Macro: Probempi vs Nonprobempi

In [17]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNonFusProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_1024mpi_1omp.GFLOPS*(dfn8MacNonFusProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_1024mpi_1omp.GFLOPS*(dfn8MacNonFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_512mpi_2omp.GFLOPS*(dfn8MacNonFusProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_512mpi_2omp.GFLOPS*(dfn8MacNonFusNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNonFusProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_256mpi_4omp.GFLOPS*(dfn8MacNonFusProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_256mpi_4omp.GFLOPS*(dfn8MacNonFusNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNonFusProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_128mpi_8omp.GFLOPS*(dfn8MacNonFusProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_128mpi_8omp.GFLOPS*(dfn8MacNonFusNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNonFusProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_64mpi_16omp.GFLOPS*(dfn8MacNonFusProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_64mpi_16omp.GFLOPS*(dfn8MacNonFusNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNonFusProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_168mpi_6omp.GFLOPS*(dfn8MacNonFusProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_168mpi_6omp.GFLOPS*(dfn8MacNonFusNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_504mpi_2omp.GFLOPS*(dfn8MacNonFusProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_504mpi_2omp.GFLOPS*(dfn8MacNonFusNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-Macro (8-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Nonfused Macro: Probempi vs Nonprobempi

In [18]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNonFusProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_1024mpi_1omp.BAND*(dfn8MacNonFusProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_1024mpi_1omp.BAND*(dfn8MacNonFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_512mpi_2omp.BAND*(dfn8MacNonFusProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_512mpi_2omp.BAND*(dfn8MacNonFusNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNonFusProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_256mpi_4omp.BAND*(dfn8MacNonFusProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_256mpi_4omp.BAND*(dfn8MacNonFusNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNonFusProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_128mpi_8omp.BAND*(dfn8MacNonFusProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_128mpi_8omp.BAND*(dfn8MacNonFusNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNonFusProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_64mpi_16omp.BAND*(dfn8MacNonFusProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_64mpi_16omp.BAND*(dfn8MacNonFusNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNonFusProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_168mpi_6omp.BAND*(dfn8MacNonFusProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_168mpi_6omp.BAND*(dfn8MacNonFusNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8MacNonFusProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusProb_504mpi_2omp.BAND*(dfn8MacNonFusProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNonFusNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNonFusNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNonFusNonProb_504mpi_2omp.BAND*(dfn8MacNonFusNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Nonfused Macro: Probempi vs Nonprobempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 






## 5) Newgeo_Macro_Probempi

### Adding Columns

In [19]:
dfn8_macro_newgeo_probempi["N_Nodes"] = 8
dfn8_macro_newgeo_probempi['NPX'] = (dfn8_macro_newgeo_probempi.GLB_X//dfn8_macro_newgeo_probempi.LOC_X)
dfn8_macro_newgeo_probempi['NPY'] = (dfn8_macro_newgeo_probempi.GLB_Y//dfn8_macro_newgeo_probempi.LOC_Y)
dfn8_macro_newgeo_probempi['NPZ'] = (dfn8_macro_newgeo_probempi.GLB_Z//dfn8_macro_newgeo_probempi.LOC_Z)
dfn8_macro_newgeo_probempi['NPT'] = (dfn8_macro_newgeo_probempi.GLB_T//dfn8_macro_newgeo_probempi.LOC_T)
dfn8_macro_newgeo_probempi["Total_proc"] = dfn8_macro_newgeo_probempi["omp_thrd"]* dfn8_macro_newgeo_probempi.MPI_size
# dfn8_macro_newgeo_probempi.columns

### Data Summary

In [20]:
dfn8_macro_newgeo_probempi_summary = dfn8_macro_newgeo_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_macro_newgeo_probempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,134
1,72,14,9,2,2,2,1008,61
2,112,9,14,2,2,2,1008,80
3,128,8,16,2,2,2,1024,70
4,144,7,18,2,2,2,1008,61
5,168,6,21,2,2,2,1008,27
6,200,5,25,2,2,2,1000,22
7,248,4,31,2,2,2,992,16
8,256,4,32,2,2,2,1024,33
9,328,3,41,2,2,2,984,12


### Cases Setup

In [21]:
dfn8MacNewgeoProb_64mpi_16omp = dfn8_macro_newgeo_probempi[(dfn8_macro_newgeo_probempi.omp_thrd == 16) & (dfn8_macro_newgeo_probempi.GLB_T/dfn8_macro_newgeo_probempi.LOC_T == 8) & (dfn8_macro_newgeo_probempi.GLB_X/dfn8_macro_newgeo_probempi.LOC_X == 2) & (dfn8_macro_newgeo_probempi.GLB_Y/dfn8_macro_newgeo_probempi.LOC_Y == 2)& (dfn8_macro_newgeo_probempi.GLB_Z/dfn8_macro_newgeo_probempi.LOC_Z == 2)]
dfn8MacNewgeoProb_128mpi_8omp = dfn8_macro_newgeo_probempi[(dfn8_macro_newgeo_probempi.omp_thrd == 8) & (dfn8_macro_newgeo_probempi.GLB_T/dfn8_macro_newgeo_probempi.LOC_T == 16) & (dfn8_macro_newgeo_probempi.GLB_X/dfn8_macro_newgeo_probempi.LOC_X == 2) & (dfn8_macro_newgeo_probempi.GLB_Y/dfn8_macro_newgeo_probempi.LOC_Y == 2)& (dfn8_macro_newgeo_probempi.GLB_Z/dfn8_macro_newgeo_probempi.LOC_Z == 2)]
dfn8MacNewgeoProb_256mpi_4omp = dfn8_macro_newgeo_probempi[(dfn8_macro_newgeo_probempi.omp_thrd == 4) & (dfn8_macro_newgeo_probempi.GLB_T/dfn8_macro_newgeo_probempi.LOC_T == 32) & (dfn8_macro_newgeo_probempi.GLB_X/dfn8_macro_newgeo_probempi.LOC_X == 2) & (dfn8_macro_newgeo_probempi.GLB_Y/dfn8_macro_newgeo_probempi.LOC_Y == 2)& (dfn8_macro_newgeo_probempi.GLB_Z/dfn8_macro_newgeo_probempi.LOC_Z == 2)]
dfn8MacNewgeoProb_512mpi_2omp = dfn8_macro_newgeo_probempi[(dfn8_macro_newgeo_probempi.omp_thrd == 2) & (dfn8_macro_newgeo_probempi.GLB_T/dfn8_macro_newgeo_probempi.LOC_T == 64) & (dfn8_macro_newgeo_probempi.GLB_X/dfn8_macro_newgeo_probempi.LOC_X == 2) & (dfn8_macro_newgeo_probempi.GLB_Y/dfn8_macro_newgeo_probempi.LOC_Y == 2)& (dfn8_macro_newgeo_probempi.GLB_Z/dfn8_macro_newgeo_probempi.LOC_Z == 2)]
dfn8MacNewgeoProb_1024mpi_1omp = dfn8_macro_newgeo_probempi[(dfn8_macro_newgeo_probempi.omp_thrd == 1) & (dfn8_macro_newgeo_probempi.GLB_T/dfn8_macro_newgeo_probempi.LOC_T == 128) & (dfn8_macro_newgeo_probempi.GLB_X/dfn8_macro_newgeo_probempi.LOC_X == 2) & (dfn8_macro_newgeo_probempi.GLB_Y/dfn8_macro_newgeo_probempi.LOC_Y == 2)& (dfn8_macro_newgeo_probempi.GLB_Z/dfn8_macro_newgeo_probempi.LOC_Z == 2)]
dfn8MacNewgeoProb_168mpi_6omp = dfn8_macro_newgeo_probempi[(dfn8_macro_newgeo_probempi.omp_thrd == 6) & (dfn8_macro_newgeo_probempi.GLB_T/dfn8_macro_newgeo_probempi.LOC_T == 21) & (dfn8_macro_newgeo_probempi.GLB_X/dfn8_macro_newgeo_probempi.LOC_X == 2) & (dfn8_macro_newgeo_probempi.GLB_Y/dfn8_macro_newgeo_probempi.LOC_Y == 2)& (dfn8_macro_newgeo_probempi.GLB_Z/dfn8_macro_newgeo_probempi.LOC_Z == 2)]
dfn8MacNewgeoProb_504mpi_2omp = dfn8_macro_newgeo_probempi[(dfn8_macro_newgeo_probempi.omp_thrd == 2) & (dfn8_macro_newgeo_probempi.GLB_T/dfn8_macro_newgeo_probempi.LOC_T == 63) & (dfn8_macro_newgeo_probempi.GLB_X/dfn8_macro_newgeo_probempi.LOC_X == 2) & (dfn8_macro_newgeo_probempi.GLB_Y/dfn8_macro_newgeo_probempi.LOC_Y == 2)& (dfn8_macro_newgeo_probempi.GLB_Z/dfn8_macro_newgeo_probempi.LOC_Z == 2)]


## 6) Newgeo_Macro_NonProbempi

### Adding Columns

In [22]:
dfn8_macro_newgeo_nonprobempi["N_Nodes"] = 8
dfn8_macro_newgeo_nonprobempi['NPX'] = (dfn8_macro_newgeo_nonprobempi.GLB_X//dfn8_macro_newgeo_nonprobempi.LOC_X)
dfn8_macro_newgeo_nonprobempi['NPY'] = (dfn8_macro_newgeo_nonprobempi.GLB_Y//dfn8_macro_newgeo_nonprobempi.LOC_Y)
dfn8_macro_newgeo_nonprobempi['NPZ'] = (dfn8_macro_newgeo_nonprobempi.GLB_Z//dfn8_macro_newgeo_nonprobempi.LOC_Z)
dfn8_macro_newgeo_nonprobempi['NPT'] = (dfn8_macro_newgeo_nonprobempi.GLB_T//dfn8_macro_newgeo_nonprobempi.LOC_T)
dfn8_macro_newgeo_nonprobempi["Total_proc"] = dfn8_macro_newgeo_nonprobempi["omp_thrd"]* dfn8_macro_newgeo_nonprobempi.MPI_size
# dfn8_macro_newgeo_nonprobempi.columns

### Data Summary

In [23]:
dfn8_macro_newgeo_nonprobempi_summary = dfn8_macro_newgeo_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_macro_newgeo_nonprobempi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,139
1,72,14,9,2,2,2,1008,61
2,112,9,14,2,2,2,1008,80
3,128,8,16,2,2,2,1024,69
4,144,7,18,2,2,2,1008,59
5,168,6,21,2,2,2,1008,27
6,200,5,25,2,2,2,1000,22
7,248,4,31,2,2,2,992,18
8,256,4,32,2,2,2,1024,34
9,328,3,41,2,2,2,984,13


### Cases Setup

In [24]:
dfn8MacNewgeoNonProb_64mpi_16omp = dfn8_macro_newgeo_nonprobempi[(dfn8_macro_newgeo_nonprobempi.omp_thrd == 16) & (dfn8_macro_newgeo_nonprobempi.GLB_T/dfn8_macro_newgeo_nonprobempi.LOC_T == 8) & (dfn8_macro_newgeo_nonprobempi.GLB_X/dfn8_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_Y/dfn8_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn8_macro_newgeo_nonprobempi.GLB_Z/dfn8_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn8MacNewgeoNonProb_128mpi_8omp = dfn8_macro_newgeo_nonprobempi[(dfn8_macro_newgeo_nonprobempi.omp_thrd == 8) & (dfn8_macro_newgeo_nonprobempi.GLB_T/dfn8_macro_newgeo_nonprobempi.LOC_T == 16) & (dfn8_macro_newgeo_nonprobempi.GLB_X/dfn8_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_Y/dfn8_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn8_macro_newgeo_nonprobempi.GLB_Z/dfn8_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn8MacNewgeoNonProb_256mpi_4omp = dfn8_macro_newgeo_nonprobempi[(dfn8_macro_newgeo_nonprobempi.omp_thrd == 4) & (dfn8_macro_newgeo_nonprobempi.GLB_T/dfn8_macro_newgeo_nonprobempi.LOC_T == 32) & (dfn8_macro_newgeo_nonprobempi.GLB_X/dfn8_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_Y/dfn8_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn8_macro_newgeo_nonprobempi.GLB_Z/dfn8_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn8MacNewgeoNonProb_512mpi_2omp = dfn8_macro_newgeo_nonprobempi[(dfn8_macro_newgeo_nonprobempi.omp_thrd == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_T/dfn8_macro_newgeo_nonprobempi.LOC_T == 64) & (dfn8_macro_newgeo_nonprobempi.GLB_X/dfn8_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_Y/dfn8_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn8_macro_newgeo_nonprobempi.GLB_Z/dfn8_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn8MacNewgeoNonProb_1024mpi_1omp = dfn8_macro_newgeo_nonprobempi[(dfn8_macro_newgeo_nonprobempi.omp_thrd == 1) & (dfn8_macro_newgeo_nonprobempi.GLB_T/dfn8_macro_newgeo_nonprobempi.LOC_T == 128) & (dfn8_macro_newgeo_nonprobempi.GLB_X/dfn8_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_Y/dfn8_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn8_macro_newgeo_nonprobempi.GLB_Z/dfn8_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn8MacNewgeoNonProb_168mpi_6omp = dfn8_macro_newgeo_nonprobempi[(dfn8_macro_newgeo_nonprobempi.omp_thrd == 6) & (dfn8_macro_newgeo_nonprobempi.GLB_T/dfn8_macro_newgeo_nonprobempi.LOC_T == 21) & (dfn8_macro_newgeo_nonprobempi.GLB_X/dfn8_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_Y/dfn8_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn8_macro_newgeo_nonprobempi.GLB_Z/dfn8_macro_newgeo_nonprobempi.LOC_Z == 2)]
dfn8MacNewgeoNonProb_504mpi_2omp = dfn8_macro_newgeo_nonprobempi[(dfn8_macro_newgeo_nonprobempi.omp_thrd == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_T/dfn8_macro_newgeo_nonprobempi.LOC_T == 63) & (dfn8_macro_newgeo_nonprobempi.GLB_X/dfn8_macro_newgeo_nonprobempi.LOC_X == 2) & (dfn8_macro_newgeo_nonprobempi.GLB_Y/dfn8_macro_newgeo_nonprobempi.LOC_Y == 2)& (dfn8_macro_newgeo_nonprobempi.GLB_Z/dfn8_macro_newgeo_nonprobempi.LOC_Z == 2)]


## GFLOPS - Newgeo Macro: Probempi vs Nonprobempi

In [25]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_1024mpi_1omp.GFLOPS*(dfn8MacNewgeoProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_1024mpi_1omp.GFLOPS*(dfn8MacNewgeoNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_512mpi_2omp.GFLOPS*(dfn8MacNewgeoProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_512mpi_2omp.GFLOPS*(dfn8MacNewgeoNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_256mpi_4omp.GFLOPS*(dfn8MacNewgeoProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_256mpi_4omp.GFLOPS*(dfn8MacNewgeoNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_128mpi_8omp.GFLOPS*(dfn8MacNewgeoProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_128mpi_8omp.GFLOPS*(dfn8MacNewgeoNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_64mpi_16omp.GFLOPS*(dfn8MacNewgeoProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_64mpi_16omp.GFLOPS*(dfn8MacNewgeoNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_168mpi_6omp.GFLOPS*(dfn8MacNewgeoProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_168mpi_6omp.GFLOPS*(dfn8MacNewgeoNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_504mpi_2omp.GFLOPS*(dfn8MacNewgeoProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_504mpi_2omp.GFLOPS*(dfn8MacNewgeoNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"Newgeo-Macro (8-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - Newgeo Macro: Probempi vs Nonprobempi

In [26]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_1024mpi_1omp.BAND*(dfn8MacNewgeoProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_1024mpi_1omp.BAND*(dfn8MacNewgeoNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNonFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_512mpi_2omp.BAND*(dfn8MacNewgeoProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_512mpi_2omp.BAND*(dfn8MacNewgeoNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_256mpi_4omp.BAND*(dfn8MacNewgeoProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_256mpi_4omp.BAND*(dfn8MacNewgeoNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_128mpi_8omp.BAND*(dfn8MacNewgeoProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_128mpi_8omp.BAND*(dfn8MacNewgeoNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_64mpi_16omp.BAND*(dfn8MacNewgeoProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_64mpi_16omp.BAND*(dfn8MacNewgeoNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_168mpi_6omp.BAND*(dfn8MacNewgeoProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_168mpi_6omp.BAND*(dfn8MacNewgeoNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8MacNewgeoProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoProb_504mpi_2omp.BAND*(dfn8MacNewgeoProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8MacNewgeoNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8MacNewgeoNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8MacNewgeoNonProb_504mpi_2omp.BAND*(dfn8MacNewgeoNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='MacNewgeoNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))



# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"Newgeo Macro: Probempi vs Nonprobempi" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 





# HIPPO AVX CASES

## 1) Fused_AVX_Probmpi

### Adding Columns

In [35]:
dfn8_avx_fused_probmpi["N_Nodes"] = 8
dfn8_avx_fused_probmpi['NPX'] = (dfn8_avx_fused_probmpi.GLB_X//dfn8_avx_fused_probmpi.LOC_X)
dfn8_avx_fused_probmpi['NPY'] = (dfn8_avx_fused_probmpi.GLB_Y//dfn8_avx_fused_probmpi.LOC_Y)
dfn8_avx_fused_probmpi['NPZ'] = (dfn8_avx_fused_probmpi.GLB_Z//dfn8_avx_fused_probmpi.LOC_Z)
dfn8_avx_fused_probmpi['NPT'] = (dfn8_avx_fused_probmpi.GLB_T//dfn8_avx_fused_probmpi.LOC_T)
dfn8_avx_fused_probmpi["Total_proc"] = dfn8_avx_fused_probmpi["omp_thrd"]* dfn8_avx_fused_probmpi.MPI_size
# dfn8_avx_fused_probmpi.columns

### Data Summary

In [36]:
dfn8_avx_fused_probmpi_summary = dfn8_avx_fused_probmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_avx_fused_probmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,139
1,72,14,9,2,2,2,1008,60
2,112,9,14,2,2,2,1008,79
3,128,8,16,2,2,2,1024,70
4,144,7,18,2,2,2,1008,60
5,168,6,21,2,2,2,1008,26
6,200,5,25,2,2,2,1000,22
7,248,4,31,2,2,2,992,18
8,256,4,32,2,2,2,1024,35
9,328,3,41,2,2,2,984,13


### Cases Setup

In [29]:
dfn8AvxFusProb_16mpi_64omp = dfn8_avx_fused_probmpi[(dfn8_avx_fused_probmpi.omp_thrd == 64) & (dfn8_avx_fused_probmpi.GLB_T/dfn8_avx_fused_probmpi.LOC_T == 2) & (dfn8_avx_fused_probmpi.GLB_X/dfn8_avx_fused_probmpi.LOC_X == 2) & (dfn8_avx_fused_probmpi.GLB_Y/dfn8_avx_fused_probmpi.LOC_Y == 2)& (dfn8_avx_fused_probmpi.GLB_Z/dfn8_avx_fused_probmpi.LOC_Z == 2)]
dfn8AvxFusProb_32mpi_32omp = dfn8_avx_fused_probmpi[(dfn8_avx_fused_probmpi.omp_thrd == 32) & (dfn8_avx_fused_probmpi.GLB_T/dfn8_avx_fused_probmpi.LOC_T == 4) & (dfn8_avx_fused_probmpi.GLB_X/dfn8_avx_fused_probmpi.LOC_X == 2) & (dfn8_avx_fused_probmpi.GLB_Y/dfn8_avx_fused_probmpi.LOC_Y == 2)& (dfn8_avx_fused_probmpi.GLB_Z/dfn8_avx_fused_probmpi.LOC_Z == 2)]
dfn8AvxFusProb_64mpi_16omp = dfn8_avx_fused_probmpi[(dfn8_avx_fused_probmpi.omp_thrd == 16) & (dfn8_avx_fused_probmpi.GLB_T/dfn8_avx_fused_probmpi.LOC_T == 8) & (dfn8_avx_fused_probmpi.GLB_X/dfn8_avx_fused_probmpi.LOC_X == 2) & (dfn8_avx_fused_probmpi.GLB_Y/dfn8_avx_fused_probmpi.LOC_Y == 2)& (dfn8_avx_fused_probmpi.GLB_Z/dfn8_avx_fused_probmpi.LOC_Z == 2)]
dfn8AvxFusProb_128mpi_8omp = dfn8_avx_fused_probmpi[(dfn8_avx_fused_probmpi.omp_thrd == 8) & (dfn8_avx_fused_probmpi.GLB_T/dfn8_avx_fused_probmpi.LOC_T == 16) & (dfn8_avx_fused_probmpi.GLB_X/dfn8_avx_fused_probmpi.LOC_X == 2) & (dfn8_avx_fused_probmpi.GLB_Y/dfn8_avx_fused_probmpi.LOC_Y == 2)& (dfn8_avx_fused_probmpi.GLB_Z/dfn8_avx_fused_probmpi.LOC_Z == 2)]
dfn8AvxFusProb_256mpi_4omp = dfn8_avx_fused_probmpi[(dfn8_avx_fused_probmpi.omp_thrd == 4) & (dfn8_avx_fused_probmpi.GLB_T/dfn8_avx_fused_probmpi.LOC_T == 32) & (dfn8_avx_fused_probmpi.GLB_X/dfn8_avx_fused_probmpi.LOC_X == 2) & (dfn8_avx_fused_probmpi.GLB_Y/dfn8_avx_fused_probmpi.LOC_Y == 2)& (dfn8_avx_fused_probmpi.GLB_Z/dfn8_avx_fused_probmpi.LOC_Z == 2)]
dfn8AvxFusProb_512mpi_2omp = dfn8_avx_fused_probmpi[(dfn8_avx_fused_probmpi.omp_thrd == 2) & (dfn8_avx_fused_probmpi.GLB_T/dfn8_avx_fused_probmpi.LOC_T == 64) & (dfn8_avx_fused_probmpi.GLB_X/dfn8_avx_fused_probmpi.LOC_X == 2) & (dfn8_avx_fused_probmpi.GLB_Y/dfn8_avx_fused_probmpi.LOC_Y == 2)& (dfn8_avx_fused_probmpi.GLB_Z/dfn8_avx_fused_probmpi.LOC_Z == 2)]
dfn8AvxFusProb_1024mpi_1omp = dfn8_avx_fused_probmpi[(dfn8_avx_fused_probmpi.omp_thrd == 1) & (dfn8_avx_fused_probmpi.GLB_T/dfn8_avx_fused_probmpi.LOC_T == 128) & (dfn8_avx_fused_probmpi.GLB_X/dfn8_avx_fused_probmpi.LOC_X == 2) & (dfn8_avx_fused_probmpi.GLB_Y/dfn8_avx_fused_probmpi.LOC_Y == 2)& (dfn8_avx_fused_probmpi.GLB_Z/dfn8_avx_fused_probmpi.LOC_Z == 2)]


## 2) Fused_AVX_NonProbmpi

### Adding Columns

In [30]:
dfn8_avx_fused_nonprobmpi["N_Nodes"] = 8
dfn8_avx_fused_nonprobmpi['NPX'] = (dfn8_avx_fused_nonprobmpi.GLB_X//dfn8_avx_fused_nonprobmpi.LOC_X)
dfn8_avx_fused_nonprobmpi['NPY'] = (dfn8_avx_fused_nonprobmpi.GLB_Y//dfn8_avx_fused_nonprobmpi.LOC_Y)
dfn8_avx_fused_nonprobmpi['NPZ'] = (dfn8_avx_fused_nonprobmpi.GLB_Z//dfn8_avx_fused_nonprobmpi.LOC_Z)
dfn8_avx_fused_nonprobmpi['NPT'] = (dfn8_avx_fused_nonprobmpi.GLB_T//dfn8_avx_fused_nonprobmpi.LOC_T)
dfn8_avx_fused_nonprobmpi["Total_proc"] = dfn8_avx_fused_nonprobmpi["omp_thrd"]* dfn8_avx_fused_nonprobmpi.MPI_size
# dfn8_avx_fused_nonprobmpi.columns

### Data Summary

In [31]:
dfn8_avx_fused_nonprobmpi_summary = dfn8_avx_fused_nonprobmpi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_avx_fused_nonprobmpi_summary

,MPI_size,omp_thrd,NPT,NPX,NPY,NPZ,Total_proc,size
0,64,16,8,2,2,2,1024,138
1,72,14,9,2,2,2,1008,62
2,112,9,14,2,2,2,1008,80
3,128,8,16,2,2,2,1024,70
4,144,7,18,2,2,2,1008,62
5,168,6,21,2,2,2,1008,27
6,200,5,25,2,2,2,1000,22
7,248,4,31,2,2,2,992,18
8,256,4,32,2,2,2,1024,35
9,328,3,41,2,2,2,984,13


### Cases Setup

In [32]:
dfn8AvxFusNonProb_64mpi_16omp = dfn8_avx_fused_nonprobmpi[(dfn8_avx_fused_nonprobmpi.omp_thrd == 16) & (dfn8_avx_fused_nonprobmpi.GLB_T/dfn8_avx_fused_nonprobmpi.LOC_T == 8) & (dfn8_avx_fused_nonprobmpi.GLB_X/dfn8_avx_fused_nonprobmpi.LOC_X == 2) & (dfn8_avx_fused_nonprobmpi.GLB_Y/dfn8_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn8_avx_fused_nonprobmpi.GLB_Z/dfn8_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn8AvxFusNonProb_128mpi_8omp = dfn8_avx_fused_nonprobmpi[(dfn8_avx_fused_nonprobmpi.omp_thrd == 8) & (dfn8_avx_fused_nonprobmpi.GLB_T/dfn8_avx_fused_nonprobmpi.LOC_T == 16) & (dfn8_avx_fused_nonprobmpi.GLB_X/dfn8_avx_fused_nonprobmpi.LOC_X == 2) & (dfn8_avx_fused_nonprobmpi.GLB_Y/dfn8_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn8_avx_fused_nonprobmpi.GLB_Z/dfn8_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn8AvxFusNonProb_256mpi_4omp = dfn8_avx_fused_nonprobmpi[(dfn8_avx_fused_nonprobmpi.omp_thrd == 4) & (dfn8_avx_fused_nonprobmpi.GLB_T/dfn8_avx_fused_nonprobmpi.LOC_T == 32) & (dfn8_avx_fused_nonprobmpi.GLB_X/dfn8_avx_fused_nonprobmpi.LOC_X == 2) & (dfn8_avx_fused_nonprobmpi.GLB_Y/dfn8_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn8_avx_fused_nonprobmpi.GLB_Z/dfn8_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn8AvxFusNonProb_512mpi_2omp = dfn8_avx_fused_nonprobmpi[(dfn8_avx_fused_nonprobmpi.omp_thrd == 2) & (dfn8_avx_fused_nonprobmpi.GLB_T/dfn8_avx_fused_nonprobmpi.LOC_T == 64) & (dfn8_avx_fused_nonprobmpi.GLB_X/dfn8_avx_fused_nonprobmpi.LOC_X == 2) & (dfn8_avx_fused_nonprobmpi.GLB_Y/dfn8_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn8_avx_fused_nonprobmpi.GLB_Z/dfn8_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn8AvxFusNonProb_1024mpi_1omp = dfn8_avx_fused_nonprobmpi[(dfn8_avx_fused_nonprobmpi.omp_thrd == 1) & (dfn8_avx_fused_nonprobmpi.GLB_T/dfn8_avx_fused_nonprobmpi.LOC_T == 128) & (dfn8_avx_fused_nonprobmpi.GLB_X/dfn8_avx_fused_nonprobmpi.LOC_X == 2) & (dfn8_avx_fused_nonprobmpi.GLB_Y/dfn8_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn8_avx_fused_nonprobmpi.GLB_Z/dfn8_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn8AvxFusNonProb_168mpi_6omp = dfn8_avx_fused_nonprobmpi[(dfn8_avx_fused_nonprobmpi.omp_thrd == 6) & (dfn8_avx_fused_nonprobmpi.GLB_T/dfn8_avx_fused_nonprobmpi.LOC_T == 21) & (dfn8_avx_fused_nonprobmpi.GLB_X/dfn8_avx_fused_nonprobmpi.LOC_X == 2) & (dfn8_avx_fused_nonprobmpi.GLB_Y/dfn8_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn8_avx_fused_nonprobmpi.GLB_Z/dfn8_avx_fused_nonprobmpi.LOC_Z == 2)]
dfn8AvxFusNonProb_504mpi_2omp = dfn8_avx_fused_nonprobmpi[(dfn8_avx_fused_nonprobmpi.omp_thrd == 2) & (dfn8_avx_fused_nonprobmpi.GLB_T/dfn8_avx_fused_nonprobmpi.LOC_T == 63) & (dfn8_avx_fused_nonprobmpi.GLB_X/dfn8_avx_fused_nonprobmpi.LOC_X == 2) & (dfn8_avx_fused_nonprobmpi.GLB_Y/dfn8_avx_fused_nonprobmpi.LOC_Y == 2)& (dfn8_avx_fused_nonprobmpi.GLB_Z/dfn8_avx_fused_nonprobmpi.LOC_Z == 2)]


## GFLOPS - AVX-Fused: Probmpi vs Non-Probempim

In [33]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8AvxFusProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_1024mpi_1omp.GFLOPS*(dfn8AvxFusProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_1024mpi_1omp.GFLOPS*(dfn8AvxFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_512mpi_2omp.Dirc_Data_Mov*(dfn8AvxFusProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_512mpi_2omp.GFLOPS*(dfn8AvxFusProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_512mpi_2omp.GFLOPS*(dfn8AvxFusNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_256mpi_4omp.Dirc_Data_Mov*(dfn8AvxFusProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_256mpi_4omp.GFLOPS*(dfn8AvxFusProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_256mpi_4omp.GFLOPS*(dfn8AvxFusNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_128mpi_8omp.Dirc_Data_Mov*(dfn8AvxFusProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_128mpi_8omp.GFLOPS*(dfn8AvxFusProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_128mpi_8omp.GFLOPS*(dfn8AvxFusNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_64mpi_16omp.Dirc_Data_Mov*(dfn8AvxFusProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_64mpi_16omp.GFLOPS*(dfn8AvxFusProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_64mpi_16omp.GFLOPS*(dfn8AvxFusNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_32mpi_32omp.Dirc_Data_Mov*(dfn8AvxFusProb_32mpi_32omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_32mpi_32omp.GFLOPS*(dfn8AvxFusProb_32mpi_32omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_32mpi_32omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-right',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_168mpi_6omp.GFLOPS*(dfn8AvxFusNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_16mpi_64omp.Dirc_Data_Mov*(dfn8AvxFusProb_16mpi_64omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_16mpi_64omp.GFLOPS*(dfn8AvxFusProb_16mpi_64omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_16mpi_64omp',
                        line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_504mpi_2omp.GFLOPS*(dfn8AvxFusNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"AVX-Fused (8-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

## BANDWIDTH - AVX-Fused: Probmpi vs Non-Probempim

In [34]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8AvxFusProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_1024mpi_1omp.BAND*(dfn8AvxFusProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_1024mpi_1omp.BAND*(dfn8AvxFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_512mpi_2omp.Dirc_Data_Mov*(dfn8AvxFusProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_512mpi_2omp.BAND*(dfn8AvxFusProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_512mpi_2omp.BAND*(dfn8AvxFusNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_256mpi_4omp.Dirc_Data_Mov*(dfn8AvxFusProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_256mpi_4omp.BAND*(dfn8AvxFusProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_256mpi_4omp.BAND*(dfn8AvxFusNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_128mpi_8omp.Dirc_Data_Mov*(dfn8AvxFusProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_128mpi_8omp.BAND*(dfn8AvxFusProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_128mpi_8omp.BAND*(dfn8AvxFusNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_64mpi_16omp.Dirc_Data_Mov*(dfn8AvxFusProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_64mpi_16omp.BAND*(dfn8AvxFusProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_64mpi_16omp.BAND*(dfn8AvxFusNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_168mpi_6omp.Dirc_Data_Mov*(dfn8AvxFusProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_168mpi_6omp.BAND*(dfn8AvxFusProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_168mpi_6omp.BAND*(dfn8AvxFusNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8AvxFusProb_504mpi_2omp.Dirc_Data_Mov*(dfn8AvxFusProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxFusProb_504mpi_2omp.BAND*(dfn8AvxFusProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxFusNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8AvxFusNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxFusNonProb_504mpi_2omp.BAND*(dfn8AvxFusNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxFusNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))


# Adding vertical and horizontal lines 
fig.add_vline(x=512e6, line_width=3, line_dash="dot", line_color="green")
fig.add_hline(y=409, line_width=3, line_dash="dash", line_color="red")

# Add annotation with arrow
fig.add_annotation(
    x='512e6'
    , y=280
    , text=f'LLC Size Per Node: 512M'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.add_annotation(
    x='130e6'
    , y=370
    , text=f'Max Bandwidth Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="orange", family="Sans Serif")
    , align="left"
    ,)


fig.update_layout(title='"AVX-Fused: Probmpi vs Non-Probempim" Bandwidth Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='Bandwidth/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"),
                  legend=go.layout.Legend(bordercolor="black",borderwidth=1)) 





NameError: name 'dfn8AvxFusProb_168mpi_6omp' is not defined

## 3) NonFused_AVX_Probempi

### Adding Columns

In [ ]:
dfn8_avx_nonfused_probempi["N_Nodes"] = 8
dfn8_avx_nonfused_probempi['NPX'] = (dfn8_avx_nonfused_probempi.GLB_X//dfn8_avx_nonfused_probempi.LOC_X)
dfn8_avx_nonfused_probempi['NPY'] = (dfn8_avx_nonfused_probempi.GLB_Y//dfn8_avx_nonfused_probempi.LOC_Y)
dfn8_avx_nonfused_probempi['NPZ'] = (dfn8_avx_nonfused_probempi.GLB_Z//dfn8_avx_nonfused_probempi.LOC_Z)
dfn8_avx_nonfused_probempi['NPT'] = (dfn8_avx_nonfused_probempi.GLB_T//dfn8_avx_nonfused_probempi.LOC_T)
dfn8_avx_nonfused_probempi["Total_proc"] = dfn8_avx_nonfused_probempi["omp_thrd"]* dfn8_avx_nonfused_probempi.MPI_size
# dfn8_avx_nonfused_probempi.columns

### Data Summary

In [ ]:
dfn8_avx_nonfused_probempi_summary = dfn8_avx_nonfused_probempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_avx_nonfused_probempi_summary

### Cases Setup

In [ ]:
dfn8AvxNonFusProb_64mpi_16omp = dfn8_avx_nonfused_probempi[(dfn8_avx_nonfused_probempi.omp_thrd == 16) & (dfn8_avx_nonfused_probempi.GLB_T/dfn8_avx_nonfused_probempi.LOC_T == 8) & (dfn8_avx_nonfused_probempi.GLB_X/dfn8_avx_nonfused_probempi.LOC_X == 2) & (dfn8_avx_nonfused_probempi.GLB_Y/dfn8_avx_nonfused_probempi.LOC_Y == 2)& (dfn8_avx_nonfused_probempi.GLB_Z/dfn8_avx_nonfused_probempi.LOC_Z == 2)]
dfn8AvxNonFusProb_128mpi_8omp = dfn8_avx_nonfused_probempi[(dfn8_avx_nonfused_probempi.omp_thrd == 8) & (dfn8_avx_nonfused_probempi.GLB_T/dfn8_avx_nonfused_probempi.LOC_T == 16) & (dfn8_avx_nonfused_probempi.GLB_X/dfn8_avx_nonfused_probempi.LOC_X == 2) & (dfn8_avx_nonfused_probempi.GLB_Y/dfn8_avx_nonfused_probempi.LOC_Y == 2)& (dfn8_avx_nonfused_probempi.GLB_Z/dfn8_avx_nonfused_probempi.LOC_Z == 2)]
dfn8AvxNonFusProb_256mpi_4omp = dfn8_avx_nonfused_probempi[(dfn8_avx_nonfused_probempi.omp_thrd == 4) & (dfn8_avx_nonfused_probempi.GLB_T/dfn8_avx_nonfused_probempi.LOC_T == 32) & (dfn8_avx_nonfused_probempi.GLB_X/dfn8_avx_nonfused_probempi.LOC_X == 2) & (dfn8_avx_nonfused_probempi.GLB_Y/dfn8_avx_nonfused_probempi.LOC_Y == 2)& (dfn8_avx_nonfused_probempi.GLB_Z/dfn8_avx_nonfused_probempi.LOC_Z == 2)]
dfn8AvxNonFusProb_512mpi_2omp = dfn8_avx_nonfused_probempi[(dfn8_avx_nonfused_probempi.omp_thrd == 2) & (dfn8_avx_nonfused_probempi.GLB_T/dfn8_avx_nonfused_probempi.LOC_T == 64) & (dfn8_avx_nonfused_probempi.GLB_X/dfn8_avx_nonfused_probempi.LOC_X == 2) & (dfn8_avx_nonfused_probempi.GLB_Y/dfn8_avx_nonfused_probempi.LOC_Y == 2)& (dfn8_avx_nonfused_probempi.GLB_Z/dfn8_avx_nonfused_probempi.LOC_Z == 2)]
dfn8AvxNonFusProb_1024mpi_1omp = dfn8_avx_nonfused_probempi[(dfn8_avx_nonfused_probempi.omp_thrd == 1) & (dfn8_avx_nonfused_probempi.GLB_T/dfn8_avx_nonfused_probempi.LOC_T == 128) & (dfn8_avx_nonfused_probempi.GLB_X/dfn8_avx_nonfused_probempi.LOC_X == 2) & (dfn8_avx_nonfused_probempi.GLB_Y/dfn8_avx_nonfused_probempi.LOC_Y == 2)& (dfn8_avx_nonfused_probempi.GLB_Z/dfn8_avx_nonfused_probempi.LOC_Z == 2)]
dfn8AvxNonFusProb_168mpi_6omp = dfn8_avx_nonfused_probempi[(dfn8_avx_nonfused_probempi.omp_thrd == 6) & (dfn8_avx_nonfused_probempi.GLB_T/dfn8_avx_nonfused_probempi.LOC_T == 21) & (dfn8_avx_nonfused_probempi.GLB_X/dfn8_avx_nonfused_probempi.LOC_X == 2) & (dfn8_avx_nonfused_probempi.GLB_Y/dfn8_avx_nonfused_probempi.LOC_Y == 2)& (dfn8_avx_nonfused_probempi.GLB_Z/dfn8_avx_nonfused_probempi.LOC_Z == 2)]
dfn8AvxNonFusProb_504mpi_2omp = dfn8_avx_nonfused_probempi[(dfn8_avx_nonfused_probempi.omp_thrd == 2) & (dfn8_avx_nonfused_probempi.GLB_T/dfn8_avx_nonfused_probempi.LOC_T == 63) & (dfn8_avx_nonfused_probempi.GLB_X/dfn8_avx_nonfused_probempi.LOC_X == 2) & (dfn8_avx_nonfused_probempi.GLB_Y/dfn8_avx_nonfused_probempi.LOC_Y == 2)& (dfn8_avx_nonfused_probempi.GLB_Z/dfn8_avx_nonfused_probempi.LOC_Z == 2)]


## 4) NonFused_AVX_NonProbempi

### Adding Columns

In [ ]:
dfn8_avx_nonfused_nonprobempi["N_Nodes"] = 8
dfn8_avx_nonfused_nonprobempi['NPX'] = (dfn8_avx_nonfused_nonprobempi.GLB_X//dfn8_avx_nonfused_nonprobempi.LOC_X)
dfn8_avx_nonfused_nonprobempi['NPY'] = (dfn8_avx_nonfused_nonprobempi.GLB_Y//dfn8_avx_nonfused_nonprobempi.LOC_Y)
dfn8_avx_nonfused_nonprobempi['NPZ'] = (dfn8_avx_nonfused_nonprobempi.GLB_Z//dfn8_avx_nonfused_nonprobempi.LOC_Z)
dfn8_avx_nonfused_nonprobempi['NPT'] = (dfn8_avx_nonfused_nonprobempi.GLB_T//dfn8_avx_nonfused_nonprobempi.LOC_T)
dfn8_avx_nonfused_nonprobempi["Total_proc"] = dfn8_avx_nonfused_nonprobempi["omp_thrd"]* dfn8_avx_nonfused_nonprobempi.MPI_size
# dfn8_avx_nonfused_nonprobempi.columns

### Data Summary

In [ ]:
dfn8_avx_nonfused_nonprobempi_summary = dfn8_avx_nonfused_nonprobempi.groupby(['MPI_size','omp_thrd','NPT','NPX', 'NPY', 'NPZ',"Total_proc"]).size().reset_index(name = 'size')
dfn8_avx_nonfused_nonprobempi_summary

### Cases Setup

In [ ]:
dfn8AvxNonFusNonProb_64mpi_16omp = dfn8_avx_nonfused_nonprobempi[(dfn8_avx_nonfused_nonprobempi.omp_thrd == 16) & (dfn8_avx_nonfused_nonprobempi.GLB_T/dfn8_avx_nonfused_nonprobempi.LOC_T == 8) & (dfn8_avx_nonfused_nonprobempi.GLB_X/dfn8_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_Y/dfn8_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_avx_nonfused_nonprobempi.GLB_Z/dfn8_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn8AvxNonFusNonProb_128mpi_8omp = dfn8_avx_nonfused_nonprobempi[(dfn8_avx_nonfused_nonprobempi.omp_thrd == 8) & (dfn8_avx_nonfused_nonprobempi.GLB_T/dfn8_avx_nonfused_nonprobempi.LOC_T == 16) & (dfn8_avx_nonfused_nonprobempi.GLB_X/dfn8_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_Y/dfn8_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_avx_nonfused_nonprobempi.GLB_Z/dfn8_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn8AvxNonFusNonProb_256mpi_4omp = dfn8_avx_nonfused_nonprobempi[(dfn8_avx_nonfused_nonprobempi.omp_thrd == 4) & (dfn8_avx_nonfused_nonprobempi.GLB_T/dfn8_avx_nonfused_nonprobempi.LOC_T == 32) & (dfn8_avx_nonfused_nonprobempi.GLB_X/dfn8_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_Y/dfn8_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_avx_nonfused_nonprobempi.GLB_Z/dfn8_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn8AvxNonFusNonProb_512mpi_2omp = dfn8_avx_nonfused_nonprobempi[(dfn8_avx_nonfused_nonprobempi.omp_thrd == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_T/dfn8_avx_nonfused_nonprobempi.LOC_T == 64) & (dfn8_avx_nonfused_nonprobempi.GLB_X/dfn8_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_Y/dfn8_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_avx_nonfused_nonprobempi.GLB_Z/dfn8_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn8AvxNonFusNonProb_1024mpi_1omp = dfn8_avx_nonfused_nonprobempi[(dfn8_avx_nonfused_nonprobempi.omp_thrd == 1) & (dfn8_avx_nonfused_nonprobempi.GLB_T/dfn8_avx_nonfused_nonprobempi.LOC_T == 128) & (dfn8_avx_nonfused_nonprobempi.GLB_X/dfn8_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_Y/dfn8_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_avx_nonfused_nonprobempi.GLB_Z/dfn8_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn8AvxNonFusNonProb_168mpi_6omp = dfn8_avx_nonfused_nonprobempi[(dfn8_avx_nonfused_nonprobempi.omp_thrd == 6) & (dfn8_avx_nonfused_nonprobempi.GLB_T/dfn8_avx_nonfused_nonprobempi.LOC_T == 21) & (dfn8_avx_nonfused_nonprobempi.GLB_X/dfn8_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_Y/dfn8_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_avx_nonfused_nonprobempi.GLB_Z/dfn8_avx_nonfused_nonprobempi.LOC_Z == 2)]
dfn8AvxNonFusNonProb_504mpi_2omp = dfn8_avx_nonfused_nonprobempi[(dfn8_avx_nonfused_nonprobempi.omp_thrd == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_T/dfn8_avx_nonfused_nonprobempi.LOC_T == 63) & (dfn8_avx_nonfused_nonprobempi.GLB_X/dfn8_avx_nonfused_nonprobempi.LOC_X == 2) & (dfn8_avx_nonfused_nonprobempi.GLB_Y/dfn8_avx_nonfused_nonprobempi.LOC_Y == 2)& (dfn8_avx_nonfused_nonprobempi.GLB_Z/dfn8_avx_nonfused_nonprobempi.LOC_Z == 2)]


## GFLOPS - NonFused-AVX: Probmpi vs Non-Probempi

In [ ]:
fig = go.Figure() 

# Case 1
fig.add_trace(go.Scatter(x=(dfn8AvxNonFusProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8AvxNonFusProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusProb_1024mpi_1omp.GFLOPS*(dfn8AvxNonFusProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_1024mpi_1omp',
                         line=dict(color='darkcyan',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxNonFusNonProb_1024mpi_1omp.Dirc_Data_Mov*(dfn8AvxNonFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusNonProb_1024mpi_1omp.GFLOPS*(dfn8AvxNonFusNonProb_1024mpi_1omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_1024mpi_1omp',
                         line=dict(color='magenta',width=3),
                         marker=dict(symbol='hexagon',size=9, line=dict(color='yellow',width=1))))


# Case 2
fig.add_trace(go.Scatter(x=(dfn8AvxNonFusProb_512mpi_2omp.Dirc_Data_Mov*(dfn8AvxNonFusProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusProb_512mpi_2omp.GFLOPS*(dfn8AvxNonFusProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_512mpi_2omp',
                         line=dict(color='royalblue',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxNonFusNonProb_512mpi_2omp.Dirc_Data_Mov*(dfn8AvxNonFusNonProb_512mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusNonProb_512mpi_2omp.GFLOPS*(dfn8AvxNonFusNonProb_512mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_512mpi_2omp',
                         line=dict(color='sandybrown',width=3),
                         marker=dict(symbol='diamond',size=5,line=dict(color='MediumPurple',width=2))))


# Case 3
fig.add_trace(go.Scatter(x=(dfn8AvxNonFusProb_256mpi_4omp.Dirc_Data_Mov*(dfn8AvxNonFusProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusProb_256mpi_4omp.GFLOPS*(dfn8AvxNonFusProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_256mpi_4omp',
                         line=dict(color='darkblue',width=3),
                         marker=dict(symbol='circle',size=7, line=dict(color='tomato',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxNonFusNonProb_256mpi_4omp.Dirc_Data_Mov*(dfn8AvxNonFusNonProb_256mpi_4omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusNonProb_256mpi_4omp.GFLOPS*(dfn8AvxNonFusNonProb_256mpi_4omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_256mpi_4omp',
                         line=dict(color='cyan',width=3),
                         marker=dict(symbol='circle',size=7,line=dict(color='Purple',width=2))))



# Case 4
fig.add_trace(go.Scatter(x=(dfn8AvxNonFusProb_128mpi_8omp.Dirc_Data_Mov*(dfn8AvxNonFusProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusProb_128mpi_8omp.GFLOPS*(dfn8AvxNonFusProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_128mpi_8omp',
                         line=dict(color='blue',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxNonFusNonProb_128mpi_8omp.Dirc_Data_Mov*(dfn8AvxNonFusNonProb_128mpi_8omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusNonProb_128mpi_8omp.GFLOPS*(dfn8AvxNonFusNonProb_128mpi_8omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_128mpi_8omp',
                         line=dict(color='red',width=3),
                         marker=dict(symbol='star',size=7, line=dict(color='black',width=1))))


# Case 5
fig.add_trace(go.Scatter(x=(dfn8AvxNonFusProb_64mpi_16omp.Dirc_Data_Mov*(dfn8AvxNonFusProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusProb_64mpi_16omp.GFLOPS*(dfn8AvxNonFusProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_64mpi_16omp',
                         line=dict(color='chartreuse',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxNonFusNonProb_64mpi_16omp.Dirc_Data_Mov*(dfn8AvxNonFusNonProb_64mpi_16omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusNonProb_64mpi_16omp.GFLOPS*(dfn8AvxNonFusNonProb_64mpi_16omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_64mpi_16omp',
                         line=dict(color='black',width=3),
                         marker=dict(symbol='square-dot',size=6, line=dict(color='mediumvioletred',width=1))))


# Case 6
fig.add_trace(go.Scatter(x=(dfn8AvxNonFusProb_168mpi_6omp.Dirc_Data_Mov*(dfn8AvxNonFusProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusProb_168mpi_6omp.GFLOPS*(dfn8AvxNonFusProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_168mpi_6omp',
                         line=dict(color='gold',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))

fig.add_trace(go.Scatter(x=(dfn8AvxNonFusNonProb_168mpi_6omp.Dirc_Data_Mov*(dfn8AvxNonFusNonProb_168mpi_6omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusNonProb_168mpi_6omp.GFLOPS*(dfn8AvxNonFusNonProb_168mpi_6omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_168mpi_6omp',
                         line=dict(color='dimgray',width=3),
                         marker=dict(symbol='triangle-up',size=6, line=dict(color='black',width=1))))



# Case 7
fig.add_trace(go.Scatter(x=(dfn8AvxNonFusProb_504mpi_2omp.Dirc_Data_Mov*(dfn8AvxNonFusProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusProb_504mpi_2omp.GFLOPS*(dfn8AvxNonFusProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusProb_504mpi_2omp',
                         line=dict(color='olive',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='olive',width=2))))

fig.add_trace(go.Scatter(x=(dfn8AvxNonFusNonProb_504mpi_2omp.Dirc_Data_Mov*(dfn8AvxNonFusNonProb_504mpi_2omp.MPI_size/8)), 
                         y=(dfn8AvxNonFusNonProb_504mpi_2omp.GFLOPS*(dfn8AvxNonFusNonProb_504mpi_2omp.MPI_size/8)), 
                         mode='lines+markers', 
                         name='AvxNonFusNonProb_504mpi_2omp',
                         line=dict(color='Brown',width=3),
                         marker=dict(symbol='circle',size=6, line=dict(color='darkred',width=2))))




# Adding a vertical line
fig.add_vline(x=512e6, line_width=3, line_dash="dash", line_color="green")
# fig.add_hline(y=1024, line_width=3, line_dash="dash", line_color="red")

# Add annotation
fig.add_annotation(
    x='512e6'
    , y=130
    , text=f'LLC Size Per Node'
    , yanchor='bottom'
    , showarrow=False
    , arrowhead=1
    , arrowsize=1
    , arrowwidth=2
    , arrowcolor="#636363"
    , ax=-20
    , ay=-30
    , font=dict(size=12, color="purple", family="Sans Serif")
    , align="left"
    ,)

fig.update_layout(title='"NonFused-AVX (8-Nodes): probempi vs nonprobempi" GFlops Measurement',
                  xaxis_title='Dirc_Data_Mov', yaxis_title='GFLOPs/Node', 
                  xaxis=dict(ticks="outside"),  
                  yaxis=dict(ticks="outside"))

In [ ]:
##5) Newgeo_AVX_Probempi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## 6) Newgeo_AVX_NonProbempi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## GFLOPS - Newgeo-AVX: Probmpi vs Non-Probempi

In [ ]:
# HIPPO VECT CASES

In [ ]:
## 1) Fused_VECT_Probmpi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## 2) Fused_VECT_NonProbmpi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## GFLOPS - Fused VECT: probempi vs nonprobempi

In [ ]:
## 3) NonFused_VECT_Probmpi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## 4) NonFused_VECT_NonProbmpi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## GFLOPS - NonFused-VECT: Probempi vs NonProbempi

In [ ]:
## 5) Newgeo_VECT_Probmpi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## 6) Newgeo_VECT_NonProbmpi

In [ ]:
### Adding Columns

In [ ]:
### Data Summary

In [ ]:
### Cases Setup

In [ ]:
## GFLOPS - Newgeo VECT: Probempi vs NonProbempi